In [3]:
import numpy as np
import skimage
import assignment3.utils as utils
import pathlib

In [90]:
def otsu_thresholding(im: np.ndarray) -> int:
    """
        Otsu's thresholding algorithm that segments an image into 1 or 0 (True or False)
        The function takes in a grayscale image and outputs a boolean image

        args:
            im: np.ndarray of shape (H, W) in the range [0, 255] (dtype=np.uint8)
        return:
            (int) the computed thresholding value
    """
    assert im.dtype == np.uint8
    ### START YOUR CODE HERE ### (You can change anything inside this block) 
    # You can also define other helper functions
    # Compute normalized histogram
    histo = np.bincount(np.reshape(im,im.shape[0]*im.shape[1]), minlength=256)
    histo = histo/(im.shape[0]*im.shape[1])
    
    # Cumulative sum
    cum_sum = np.cumsum(histo)
    # Cumulative mean
    cum_mean = np.empty(256) 
    cum_mean[0] = 0.0
    for i in range(1,256):
        cum_mean[i] = cum_mean[i-1] + i * histo[i]
    
    # Total mean
    glob_mean = cum_mean[255]
    
    # Betewwn classes variance
    sigma_b = np.square(glob_mean*cum_sum - cum_mean)/(cum_sum*(1-cum_sum))
    
    sigma_b[np.isnan(sigma_b)] = 0.
    
    max_sigma = np.amax(sigma_b)
    
    # The values that has the highest sigma are averaged
    threshold = round(np.mean(np.where(sigma_b == max_sigma)))
    
    return threshold
    ### END YOUR CODE HERE ###

In [91]:
if __name__ == "__main__":
    # DO NOT CHANGE
    impaths_to_segment = [
        pathlib.Path("thumbprint.png"),
        pathlib.Path("polymercell.png")
    ]
    for impath in impaths_to_segment:
        im = utils.read_image(impath)
        threshold = otsu_thresholding(im)
        print("Found optimal threshold:", threshold)

        # Segment the image by threshold
        segmented_image = (im >= threshold)
        assert im.shape == segmented_image.shape, "Expected image shape ({}) to be same as thresholded image shape ({})".format(
                im.shape, segmented_image.shape)
        assert segmented_image.dtype == np.bool, "Expected thresholded image dtype to be np.bool. Was: {}".format(
                segmented_image.dtype)

        segmented_image = utils.to_uint8(segmented_image)

        save_path = "{}-segmented.png".format(impath.stem)
        utils.save_im(save_path, segmented_image)

Reading image: images/thumbprint.png
Found optimal threshold: 153
Saving image to: image_processed/thumbprint-segmented.png


/Users/nicolo/opt/anaconda3/envs/tdt4195/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide
/Users/nicolo/opt/anaconda3/envs/tdt4195/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  app.launch_new_instance()


Reading image: images/polymercell.png
Found optimal threshold: 181
Saving image to: image_processed/polymercell-segmented.png
